In [41]:
from sklearn import datasets
import numpy as np
import pandas as pd
from pandasql import sqldf
from jupyterutils.togglecode import hideCode
%matplotlib inline
pysqldf = lambda q: sqldf(q, globals())

In [42]:
from sklearn import datasets

In [43]:
data = datasets.load_iris()
y = data.target
y[y > 1] = 0
y

x = data.data[:, 0]
x
w = np.ones(x.shape)

In [44]:
pd.DataFrame(data.data).to_csv("../resources/iris_x.csv", index=False, header=False)

In [45]:
pd.DataFrame(y).to_csv("../resources/iris_y.csv", index=False, header=False)

In [46]:
eps = np.finfo(float).eps

In [47]:
# val y = Array[Double](1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0)
# val x = Array[Double](53,56,57,63,66,67,67,67,68,69,70,70,70,70,72,73,75,75,76,76,78,79,80,81)
# val w = DenseVector.fill(24){1/24d}.toArray

In [48]:
# y = np.array([1,1,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,1,0,0,0,0,0,0])
# x = np.array([53,56,57,63,66,67,67,67,68,69,70,70,70,70,72,73,75,75,76,76,78,79,80,81])
# w = np.ones(24)/24
#w = np.array([0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.20833333333333337,0.20833333333333337,0.20833333333333337,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.20833333333333337,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664,0.041666666666666664])

In [49]:
data = pd.DataFrame([x,y,w]).T
data.columns = ["x", "y", "w"]
data

x    y    w
0    5.1  0.0  1.0
1    4.9  0.0  1.0
2    4.7  0.0  1.0
3    4.6  0.0  1.0
4    5.0  0.0  1.0
5    5.4  0.0  1.0
6    4.6  0.0  1.0
7    5.0  0.0  1.0
8    4.4  0.0  1.0
9    4.9  0.0  1.0
10   5.4  0.0  1.0
11   4.8  0.0  1.0
12   4.8  0.0  1.0
13   4.3  0.0  1.0
14   5.8  0.0  1.0
15   5.7  0.0  1.0
16   5.4  0.0  1.0
17   5.1  0.0  1.0
18   5.7  0.0  1.0
19   5.1  0.0  1.0
20   5.4  0.0  1.0
21   5.1  0.0  1.0
22   4.6  0.0  1.0
23   5.1  0.0  1.0
24   4.8  0.0  1.0
25   5.0  0.0  1.0
26   5.0  0.0  1.0
27   5.2  0.0  1.0
28   5.2  0.0  1.0
29   4.7  0.0  1.0
..   ...  ...  ...
120  6.9  0.0  1.0
121  5.6  0.0  1.0
122  7.7  0.0  1.0
123  6.3  0.0  1.0
124  6.7  0.0  1.0
125  7.2  0.0  1.0
126  6.2  0.0  1.0
127  6.1  0.0  1.0
128  6.4  0.0  1.0
129  7.2  0.0  1.0
130  7.4  0.0  1.0
131  7.9  0.0  1.0
132  6.4  0.0  1.0
133  6.3  0.0  1.0
134  6.1  0.0  1.0
135  7.7  0.0  1.0
136  6.3  0.0  1.0
137  6.4  0.0  1.0
138  6.0  0.0  1.0
139  6.9  0.0  1.0
140  6.7  0.0  1.0
141  6.9  0.0  1.0
142  5.8  0.0  1.0
143  6.8  0.0  1.0
144  6.7  0.0  1.0
145  6.7  0.0  1.0
146  6.3  0.0  1.0
147  6.5  0.0  1.0
148  6.2  0.0  1.0
149  5.9  0.0  1.0

[150 rows x 3 columns]

In [50]:
p_overall = (data["y"]* data["w"]).sum() / data["w"].sum()
n_overall = data["w"].sum()
y_overall = (data["y"] * data["w"]).sum()

In [51]:
# p = (data["y"]* data["w"]).sum() / data["w"].sum()
p_overall

0.3333333333333333

In [52]:
w = data["w"].sum()

data_out = pysqldf("""select x, sum(y*w) as y, sum(w) as n from data group by x""")

data_out["y_left_cuml"] = data_out["y"].cumsum().clip(0)
data_out["n_left_cuml"] = data_out["n"].cumsum().clip(0)
data_out["p_left"] = data_out["y_left_cuml"] / data_out["n_left_cuml"]

data_out["y_right_cuml"] = y_overall - data_out["y"].cumsum().clip(0)
data_out["n_right_cuml"] = n_overall - data_out["n_left_cuml"]
data_out["p_right"] = (data_out["y_right_cuml"] /  data_out["n_right_cuml"])

In [54]:
data_out[ data_out < eps ] = 0
data_out.head(10)

x    y     n  y_left_cuml  n_left_cuml    p_left  y_right_cuml  \
0  4.3  0.0   1.0          0.0          1.0  0.000000          50.0   
1  4.4  0.0   3.0          0.0          4.0  0.000000          50.0   
2  4.5  0.0   1.0          0.0          5.0  0.000000          50.0   
3  4.6  0.0   4.0          0.0          9.0  0.000000          50.0   
4  4.7  0.0   2.0          0.0         11.0  0.000000          50.0   
5  4.8  0.0   5.0          0.0         16.0  0.000000          50.0   
6  4.9  1.0   6.0          1.0         22.0  0.045455          49.0   
7  5.0  2.0  10.0          3.0         32.0  0.093750          47.0   
8  5.1  1.0   9.0          4.0         41.0  0.097561          46.0   
9  5.2  1.0   4.0          5.0         45.0  0.111111          45.0   

   n_right_cuml   p_right  
0         149.0  0.335570  
1         146.0  0.342466  
2         145.0  0.344828  
3         141.0  0.354610  
4         139.0  0.359712  
5         134.0  0.373134  
6         128.0  0.382812  
7         118.0  0.398305  
8         109.0  0.422018  
9         105.0  0.428571

In [55]:
data["y"].sum() / data["y"].count()

0.3333333333333333

In [56]:
def entropy(p):
    return 0 if p == 1 or p == 0 else -p*np.log(p) /np.log(2) - (1-p)*np.log(1-p) /np.log(2)

In [57]:

data_out["y_right_cuml"].clip(0, inplace=True)
data_out["y_left_cuml"].clip(0, inplace=True)


In [58]:
pl = data_out["p_left"]
pr = data_out["p_right"]

data_out["entropy_left"] = data_out["p_left"].apply(entropy)
data_out["entropy_right"] = data_out["p_right"].apply(entropy)

In [59]:
m,n = data.shape
data_out["weighted_entropy"]= data_out["entropy_left"]*data_out["n_left_cuml"] + data_out["entropy_right"]*data_out["n_right_cuml"]

In [60]:
data_out["weighted_entropy"] /= w.sum()
data_out.head(10)

x    y     n  y_left_cuml  n_left_cuml    p_left  y_right_cuml  \
0  4.3  0.0   1.0          0.0          1.0  0.000000          50.0   
1  4.4  0.0   3.0          0.0          4.0  0.000000          50.0   
2  4.5  0.0   1.0          0.0          5.0  0.000000          50.0   
3  4.6  0.0   4.0          0.0          9.0  0.000000          50.0   
4  4.7  0.0   2.0          0.0         11.0  0.000000          50.0   
5  4.8  0.0   5.0          0.0         16.0  0.000000          50.0   
6  4.9  1.0   6.0          1.0         22.0  0.045455          49.0   
7  5.0  2.0  10.0          3.0         32.0  0.093750          47.0   
8  5.1  1.0   9.0          4.0         41.0  0.097561          46.0   
9  5.2  1.0   4.0          5.0         45.0  0.111111          45.0   

   n_right_cuml   p_right  entropy_left  entropy_right  weighted_entropy  
0         149.0  0.335570      0.000000       0.920517          0.914380  
1         146.0  0.342466      0.000000       0.927159          0.902435  
2         145.0  0.344828      0.000000       0.929364          0.898385  
3         141.0  0.354610      0.000000       0.938118          0.881831  
4         139.0  0.359712      0.000000       0.942444          0.873331  
5         134.0  0.373134      0.000000       0.953048          0.851390  
6         128.0  0.382812      0.266765       0.960004          0.858329  
7         118.0  0.398305      0.448864       0.969950          0.858785  
8         109.0  0.422018      0.461216       0.982382          0.839930  
9         105.0  0.428571      0.503258       0.985228          0.840637

In [61]:
entropy_overall = entropy(p_overall)

In [62]:
entropy_overall

0.9182958340544896

In [63]:
data_out["gain"] = entropy_overall - data_out["weighted_entropy"]

In [70]:
data_out.head(20)

x    y     n  y_left_cuml  n_left_cuml    p_left  y_right_cuml  \
0   4.3  0.0   1.0          0.0          1.0  0.000000          50.0   
1   4.4  0.0   3.0          0.0          4.0  0.000000          50.0   
2   4.5  0.0   1.0          0.0          5.0  0.000000          50.0   
3   4.6  0.0   4.0          0.0          9.0  0.000000          50.0   
4   4.7  0.0   2.0          0.0         11.0  0.000000          50.0   
5   4.8  0.0   5.0          0.0         16.0  0.000000          50.0   
6   4.9  1.0   6.0          1.0         22.0  0.045455          49.0   
7   5.0  2.0  10.0          3.0         32.0  0.093750          47.0   
8   5.1  1.0   9.0          4.0         41.0  0.097561          46.0   
9   5.2  1.0   4.0          5.0         45.0  0.111111          45.0   
10  5.3  0.0   1.0          5.0         46.0  0.108696          45.0   
11  5.4  1.0   6.0          6.0         52.0  0.115385          44.0   
12  5.5  5.0   7.0         11.0         59.0  0.186441          39.0   
13  5.6  5.0   6.0         16.0         65.0  0.246154          34.0   
14  5.7  5.0   8.0         21.0         73.0  0.287671          29.0   
15  5.8  3.0   7.0         24.0         80.0  0.300000          26.0   
16  5.9  2.0   3.0         26.0         83.0  0.313253          24.0   
17  6.0  4.0   6.0         30.0         89.0  0.337079          20.0   
18  6.1  4.0   6.0         34.0         95.0  0.357895          16.0   
19  6.2  2.0   4.0         36.0         99.0  0.363636          14.0   

    n_right_cuml   p_right  entropy_left  entropy_right  weighted_entropy  \
0          149.0  0.335570      0.000000       0.920517          0.914380   
1          146.0  0.342466      0.000000       0.927159          0.902435   
2          145.0  0.344828      0.000000       0.929364          0.898385   
3          141.0  0.354610      0.000000       0.938118          0.881831   
4          139.0  0.359712      0.000000       0.942444          0.873331   
5          134.0  0.373134      0.000000       0.953048          0.851390   
6          128.0  0.382812      0.266765       0.960004          0.858329   
7          118.0  0.398305      0.448864       0.969950          0.858785   
8          109.0  0.422018      0.461216       0.982382          0.839930   
9          105.0  0.428571      0.503258       0.985228          0.840637   
10         104.0  0.432692      0.495969       0.986888          0.836340   
11          98.0  0.448980      0.515947       0.992476          0.827279   
12          91.0  0.428571      0.693966       0.985228          0.870665   
13          85.0  0.400000      0.805125       0.970951          0.899093   
14          77.0  0.376623      0.865694       0.955622          0.911857   
15          70.0  0.371429      0.881291       0.951763          0.914178   
16          67.0  0.358209      0.896893       0.941186          0.916677   
17          61.0  0.327869      0.921996       0.912734          0.918229   
18          55.0  0.290909      0.940922       0.869893          0.914878   
19          51.0  0.274510      0.945660       0.847862          0.912409   

        gain  
0   0.003916  
1   0.015861  
2   0.019911  
3   0.036465  
4   0.044964  
5   0.066906  
6   0.059967  
7   0.059510  
8   0.078366  
9   0.077659  
10  0.081956  
11  0.091017  
12  0.047631  
13  0.019203  
14  0.006439  
15  0.004118  
16  0.001618  
17  0.000066  
18  0.003418  
19  0.005887

In [68]:
data_out["gain"].idxmax()

11

In [69]:
data_out.loc[11]

x                    5.400000
y                    1.000000
n                    6.000000
y_left_cuml          6.000000
n_left_cuml         52.000000
p_left               0.115385
y_right_cuml        44.000000
n_right_cuml        98.000000
p_right              0.448980
entropy_left         0.515947
entropy_right        0.992476
weighted_entropy     0.827279
gain                 0.091017
Name: 11, dtype: float64